## ETL de Carga de información de Datos Maestros VF.xlsx

## Installation of base libraries

In [2]:
import pandas as pd
from config.db_config import engine
import re

## Extraction

In [3]:
# Se extrae el primer source
df_datos_maestros = pd.read_excel("../data/Datos Maestros VF.xlsx")

In [4]:
# Se extrae el segundo source
df_ddec = pd.read_csv(
    "../data/dDEC1204.TXT", delimiter=",", header=None, encoding="latin1"
)

## Transformation

In [5]:
# Renombrado de columnas
columnas = [
    "Nombre visible Agente",
    "AGENTE (OFEI)",
    "CENTRAL (dDEC, dSEGDES, dPRU…)",
    "Tipo de central (Hidro, Termo, Filo, Menor)",
]
df_datos_maestros = pd.DataFrame(df_datos_maestros, columns=columnas)
df_datos_maestros = df_datos_maestros.rename(
    columns={
        "Nombre visible Agente": "Nombre_Agente",
        "AGENTE (OFEI)": "Agente_Ofei",
        "CENTRAL (dDEC, dSEGDES, dPRU…)": "Central",
        "Tipo de central (Hidro, Termo, Filo, Menor)": "Tipo_Central",
    }
)
# Filtros solicitados
condicion = (
    (df_datos_maestros["Agente_Ofei"] == "EMGESA")
    | (df_datos_maestros["Agente_Ofei"] == "EMGESA S.A.")
) & (
    (df_datos_maestros["Tipo_Central"] == "H")
    | (df_datos_maestros["Tipo_Central"] == "T")
)
df_filtrado = df_datos_maestros[condicion]
df_datos_maestros = df_filtrado

,Nombre_Agente,Agente_Ofei,Central,Tipo_Central
32,EMGESA,EMGESA S.A.,BETANIA,H
33,EMGESA,EMGESA S.A.,BETANIA,H
34,EMGESA,EMGESA S.A.,BETANIA,H
49,EMGESA,EMGESA S.A.,CTGEMG1,T
50,EMGESA,EMGESA S.A.,CTGEMG2,T


In [6]:
# Renombrado de columnas
num_columnas_valores = len(df_ddec.columns) - 1
nombres_columnas = ["Central"] + [f"Hora_{i+1}" for i in range(num_columnas_valores)]

df_ddec.columns = nombres_columnas

df_ddec = df_ddec.copy()

,Central,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,Hora_7,Hora_8,Hora_9,...,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24
0,AG_ARGOSCTG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AG_ARGOSYUM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AG_ELCAIRO,0,0,0,1,3,3,3,3,3,...,3,0,0,0,0,1,3,3,3,3
3,AG_REFICAR,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4,AG_UNIBOL,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


#### Merge

In [8]:
df_merge = df_datos_maestros.merge(
    df_ddec, left_on="Central", right_on="Central", how="inner"
)
# Elimino duplicados del merge
df_sin_duplicados = df_merge.drop_duplicates()

,Nombre_Agente,Agente_Ofei,Central,Tipo_Central,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,364
3,EMGESA,EMGESA S.A.,CTGEMG1,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EMGESA,EMGESA S.A.,CTGEMG2,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,EMGESA,EMGESA S.A.,CTGEMG3,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
8,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,180,120,120,997,1002,780,140,0,0
13,EMGESA,EMGESA S.A.,PAGUA,H,600,600,600,600,600,600,...,600,600,600,600,600,600,600,600,600,600
19,EMGESA,EMGESA S.A.,ZIPAEMG2,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,EMGESA,EMGESA S.A.,ZIPAEMG3,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,EMGESA,EMGESA S.A.,ZIPAEMG4,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Selecciono las columnas y creo el campo con la suma total
df_suma_total = df_sin_duplicados.loc[:, "Hora_1":]
df_suma_total["Total_Horas"] = df_suma_total.sum(axis=1)

# Conservo las columnas 'Nombre_Agente', 'Agente_Ofei', 'Central' y agrego la columna 'Total_Horas'
df_final = pd.concat(
    [df_sin_duplicados[["Nombre_Agente", "Agente_Ofei", "Central"]], df_suma_total],
    axis=1,
)

# Limpieza de campo "Agente_Ofei"
df_final["Agente_Ofei"] = df_final["Agente_Ofei"].apply(lambda x: re.sub(r'[^a-zA-Z\s]', "", x))

In [28]:
# Selección de columnas > 0
datos_mayor_cero = (df_final['Total_Horas'] > 0)
df_final = df_final[datos_mayor_cero]

In [29]:
# Creación del Id
df_final["Id"] = range(1, len(df_final) + 1)

In [30]:
# Selección de columnas finales
columnas_a = ['Id', 'Nombre_Agente', 'Agente_Ofei', 'Central']
# columnas_b = df_final.loc[:, 'Hora_1':'Total_Horas'].astype('float')
columnas_b = df_final.loc[:, 'Total_Horas':'Total_Horas'].astype('float')
df_final = pd.concat([df_final[columnas_a], columnas_b], axis=1)

## Load

In [31]:
def carga_df_tabla(dataframe, engine, table_name):
    try:
        conn = engine.connect()
        columns = dataframe.columns.tolist()
        dataframe.to_sql(name=table_name, con=engine, if_exists="replace", index=False)
        print(f"Los datos se han cargado correctamente en la tabla {table_name}")
    except Exception as e:
        print(f"Error al cargar datos desde el DataFrame a la tabla {table_name}: {e}")
    finally:
        conn.close()

In [32]:
carga_df_tabla(df_final, engine, "tbl_datos_agente")

Los datos se han cargado correctamente en la tabla tbl_datos_agente
